## Loading sql and the Data Warehouse

In [1]:
%load_ext sql
%sql sqlite:///CourseDataWarehouse.db

'Connected: @CourseDataWarehouse.db'

### Using SQL's 'attach' function to import the original CourseData.db to this notebook

In [2]:
%%sql
attach "CourseData.db" as reldb

 * sqlite:///CourseDataWarehouse.db
Done.


[]

### Dropping tables to avoid duplicates

In [3]:
%%sql 
DROP TABLE IF EXISTS PROGRAMS_WH;
DROP TABLE IF EXISTS LOCATIONS_WH;
DROP TABLE IF EXISTS TIMES_WH;
DROP TABLE IF EXISTS FACULTY_WH;
DROP TABLE IF EXISTS CATALOGS_WH;
DROP TABLE IF EXISTS COURSES_WH;

 * sqlite:///CourseDataWarehouse.db
Done.
Done.
Done.
Done.
Done.
Done.


[]

## Creating the Dimension tables for the Star Schema

In [4]:
%%sql
CREATE TABLE PROGRAMS_WH (
    PID INTEGER NOT NULL PRIMARY KEY,
    program_name TEXT NOT NULL,
    program_code VARCHAR NOT NULL
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [5]:
%%sql
CREATE TABLE LOCATIONS_WH (
    LID INTEGER PRIMARY KEY,
    location VARCHAR
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [6]:
%%sql 
CREATE TABLE FACULTY_WH (
    FID  INTEGER NOT NULL PRIMARY KEY,
    FName TEXT NOT NULL
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [7]:
%%sql
CREATE TABLE CATALOGS_WH (
    CatID INTEGER PRIMARY KEY,
    CourseCode VARCHAR NOT NULL,
    Credits TEXT NOT NULL,
    Prereqs TEXT,
    Coreqs TEXT,
    Fees INTEGER,
    Description TEXT,
    Attributes TEXT
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

### Creating the fact table of the Star Schema (COURSES)

In [8]:
%%sql
CREATE TABLE COURSES_WH (
    CID INTEGER PRIMARY KEY,
    CRN INTEGER NOT NULL,
    term TEXT NOT NULL,
    Title TEXT NOT NULL,
    Section VARCHAR NOT NULL,
    Act INTEGER,
    Cap INTEGER,
    Rem INTEGER, 
    timecodes TEXT,
    PID INTEGER NOT NULL,
    LID INTEGER NOT NULL,
    FID INTEGER NOT NULL,
    CatID INTEGER NOT NULL,
    FOREIGN KEY (PID) REFERENCES PROGRAMS_WH (PID),
    FOREIGN KEY (LID) REFERENCES LOCATIONS_WH (LID),
    FOREIGN KEY (FID) REFERENCES FACULTY_WH (FID),
    FOREIGN KEY (CatID) REFERENCES CATALOGS_WH (CatID)
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

### Inserting the data into the Warehouse tables using the CourseData.db proxy table 'reldb'

In [9]:
%%sql

INSERT INTO PROGRAMS_WH (program_name, program_code)
SELECT DISTINCT program_name, program_code
FROM reldb.PROGRAMS;

 * sqlite:///CourseDataWarehouse.db
83 rows affected.


[]

In [10]:
%%sql

INSERT INTO LOCATIONS_WH (location)
SELECT DISTINCT location
FROM reldb.COURSE_MEETINGS;

 * sqlite:///CourseDataWarehouse.db
201 rows affected.


[]

In [11]:
%%sql

INSERT INTO FACULTY_WH (FName)
SELECT DISTINCT FName
FROM reldb.FACULTY;

 * sqlite:///CourseDataWarehouse.db
1104 rows affected.


[]

In [12]:
%%sql

INSERT INTO CATALOGS_WH (CourseCode, credits, Prereqs, Coreqs, Fees, Description, Attributes)
SELECT DISTINCT CourseCode, credits, prereqs, coreqs, fees, description, attributes
FROM reldb.CATALOGS;

 * sqlite:///CourseDataWarehouse.db
2220 rows affected.


[]

In [13]:
%%sql

INSERT INTO COURSES_WH (CRN, term, Title, Section, Act, Cap, Rem, timecodes, PID, LID, FID, CatID)
SELECT DISTINCT CRN, reldb.COURSES.term, title, section, act, cap, rem, timecodes, PID, LID, FID, CatID
FROM reldb.COURSES
    JOIN reldb.CATALOGS USING (CourseCode)
    JOIN PROGRAMS_WH USING (PID)
    JOIN reldb.COURSE_MEETINGS USING (CRN)
    JOIN LOCATIONS_WH USING (location)
    JOIN FACULTY_WH USING (FID)
    JOIN CATALOGS_WH USING (CatID);

 * sqlite:///CourseDataWarehouse.db
17886 rows affected.


[]

In [14]:
%%sql
vacuum;

 * sqlite:///CourseDataWarehouse.db
Done.


[]